In [1]:
import pandas_datareader.data as web
import datetime
from decimal import Decimal
from dateutil import relativedelta
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, plot_heatmaps, resample_apply
import talib as ta
import importlib
import os
import platform
import psutil
import numpy as np

In [2]:
start = datetime.datetime.today() + relativedelta.relativedelta(years=-1)
end = datetime.date.today()
# data = web.DataReader('1357.T', 'yahoo', start, end)  #NEXT FUNDS 日経平均ダブルインバース・インデックス連動型上場投信
data = web.DataReader('1570.T', 'yahoo', start, end)  #NEXT FUNDS 日経平均レバレッジ・インデックス連動型上場投信
data = data.astype("double")

'''
!!!ここでカスタムロジックでSignal列に売り買い用のSignal値を入れる
'''
data["Signal"] = np.random.randint(-1, 2, len(data))

print(data)
initial = 1000000
fname = f"html/plot-"


               High      Low     Open    Close     Volume  Adj Close  Signal
Date                                                                        
2021-09-13  17510.0  17240.0  17400.0  17510.0  8217909.0    17510.0       0
2021-09-14  17920.0  17570.0  17660.0  17760.0  9275477.0    17760.0       0
2021-09-15  17670.0  17390.0  17520.0  17570.0  7529060.0    17570.0       0
2021-09-16  17720.0  17230.0  17710.0  17360.0  8158329.0    17360.0      -1
2021-09-17  17620.0  17400.0  17450.0  17570.0  5910472.0    17570.0       0
...             ...      ...      ...      ...        ...        ...     ...
2022-09-07  14155.0  13860.0  14140.0  14045.0  9128578.0    14045.0       0
2022-09-08  14700.0  14340.0  14345.0  14685.0  9606072.0    14685.0       0
2022-09-09  14925.0  14710.0  14850.0  14865.0  9063140.0    14865.0       0
2022-09-12  15270.0  15080.0  15165.0  15170.0  6594283.0    15170.0       0
2022-09-13  15320.0  15180.0  15230.0  15280.0  6524247.0    15280.0       0

In [3]:
'''
Signalはdataのほうに入っているから、クラスは固定でいい
'''
class SignalStrategy(Strategy):
    def init(self):
        pass

    def next(self):
        current_signal = self.data.Signal[-1]

        if current_signal == 1:
            if not self.position:
                self.buy()
        elif current_signal == -1:
            if self.position:
                self.position.close()

In [4]:
bt = Backtest(
    data,
    SignalStrategy,
    cash = initial,
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False) # True：現在の終値で取引，False：次の時間の始値で取引)

output = bt.run()
print(output) # 実行結果(データ)
display(output._trades)
bt.plot(filename = fname + str(output._strategy) + ".html")

Start                     2021-09-13 00:00:00
End                       2022-09-13 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                   74.796748
Equity Final [$]                  470100.9185
Equity Peak [$]                     1000000.0
Return [%]                         -52.989908
Buy & Hold Return [%]               -12.73558
Return (Ann.) [%]                  -53.847444
Volatility (Ann.) [%]               15.146034
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -52.989908
Avg. Drawdown [%]                  -52.989908
Max. Drawdown Duration      354 days 00:00:00
Avg. Drawdown Duration      354 days 00:00:00
# Trades                                   42
Win Rate [%]                        26.190476
Best Trade [%]                      18.646671
Worst Trade [%]                    -14.460458
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,57,8,11,17385.63500,16710.0,-38511.19500,-0.038862,2021-09-27,2021-09-30,3 days
1,58,12,25,16420.88300,16440.0,1108.78600,0.001164,2021-10-01,2021-10-20,19 days
2,59,26,31,16209.84350,15770.0,-25950.76650,-0.027134,2021-10-21,2021-10-28,7 days
3,56,33,34,16551.52650,16530.0,-1205.48400,-0.001301,2021-11-01,2021-11-02,1 days
4,55,37,40,16883.16000,15990.0,-49123.80000,-0.052902,2021-11-08,2021-11-11,3 days
5,52,44,47,17023.85300,16620.0,-21000.35600,-0.023723,2021-11-17,2021-11-22,5 days
6,52,49,51,16511.32850,15280.0,-64029.08200,-0.074575,2021-11-25,2021-11-29,4 days
7,51,52,56,15626.97250,14805.0,-41920.59750,-0.052600,2021-11-30,2021-12-06,6 days
8,48,58,62,15782.73975,15375.0,-19571.50800,-0.025835,2021-12-08,2021-12-14,6 days
9,46,64,66,15943.53175,15090.0,-39262.46050,-0.053535,2021-12-16,2021-12-20,4 days


Row(id='1407', ...)